In [1]:
import requests
from bs4 import BeautifulSoup
import time
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt

delay = 3
browser = Chrome("./chromedriver")
browser.implicitly_wait(delay)
browser.get("https://www.youtube.com/channel/UCdM-fLpO0Nv67NLDofSl9yA/videos") # 박막례할머니 동영상 페이지로 들어감
browser.maximize_window()
body = browser.find_element_by_tag_name('body') # body태그 가져오기

num_of_pagedowns = 22
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1 #video페이지의 스크롤을 충분히 밑으로 내려놓고

In [2]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')
video_list2 = html.select('div #dismissable')

youtube_url = 'http://www.youtube.com'

park_url = []

#썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요 를 저장하기 위한 리스트
thumbnail_list = []
category_list = []
upload_date_list = []
video_length_list = []
title_list = []
view_count_list = []
likes_count_list = []
dislikes_count_list = []


print("video_list2의 길이:",len(video_list2))
for i in range(len(video_list2)):
    url = youtube_url + video_list2[i].find('a',{'id': 'thumbnail'})['href']
    park_url.append(url) # 모든 영상의 url을 youtube_url에 붙여서 완전한 url로 만들고 park_url이라는 리스트에 넣어줌

video_list2의 길이: 96


In [3]:
for video in video_list2:
    video_length = video.select_one("ytd-thumbnail-overlay-time-status-renderer.ytd-thumbnail > span.style-scope")
    video_length = video_length.text.replace("\n","").strip()
    video_length_list.append(video_length)
    print(video_length)

8:43
11:39
7:37
12:02
14:46
15:42
18:41
15:12
11:11
10:26
10:59
15:54
10:16
8:08
18:10
6:30
9:18
10:16
7:15
9:16
6:46
12:12
12:05
10:01
17:54
17:22
7:01
8:41
6:38
8:21
9:44
3:43
6:19
10:13
11:35
8:01
7:15
8:15
7:16
8:25
7:14
55:20
16:05
20:25
12:53
13:16
15:02
7:25
1:08:06
8:31
7:46
15:27
16:09
11:21
12:35
10:10
6:37
13:35
9:28
11:20
54:04
11:13
15:04
15:28
7:20
7:39
12:56
12:00
14:38
13:45
14:34
10:17
7:29
15:08
12:35
12:27
9:15
13:16
17:23
10:33
10:12
6:47
8:42
13:01
7:45
13:44
11:30
12:57
9:42
5:58
7:56
6:42
5:20
11:57
3:21
2:01


In [4]:
# 현재로서는 썸네일, 유형, 동영상 게시일, 영상길이, 제목, 조회수, 좋아요, 싫어요, 만 크롤링 하면 될듯!

# park = pd.DataFrame({'thumbnail': [],
#                      'title' : [],
#                      'category': [],
#                      'view_count': [],
#                      'likes_count': [],
#                      'dislikes_count':[],
#                      'video_length':[],
#                      'upload_date': [],
#                      }) #총 8개의 정보

In [ ]:
from selenium.common.exceptions import ElementNotInteractableException

for i in range(len(video_list2)):
    try:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
                                                        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except ElementNotInteractableException:

        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        # browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        # category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # category = html.find('div', {'id':'always-shown'}).find('div',{'id':'content'}).find("a":{"class":"style-scope"}).text
        category = html.select_one("div#always-shown div#content a.style-scope")
        
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        'aria-label':re.compile('좋아요')}).text+'개'
        dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        'aria-label':re.compile('싫어요')}).text+'개'
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8

        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    except AttributeError:
        #여기까지는 video페이지에서 수집할 수 있는 썸네일, 제목을 가져온다.
        thumbnail = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src'] # 그 하위의 img라는 태그를 또 찾고 거기서 src가져와라 # 1
        title = video_list2[i].find('a',{'id':'video-title'}).text # 2 

        # 여기서부터 영상 클릭해서 들어간 페이지
        click_url = park_url[i]
        browser.get(click_url)
        time.sleep(1.5) # 페이지를 넘어가야하므로 로딩을 기다려야함
        # body = browser.find_element_by_tag_name('body')
        browser.find_element_by_xpath("//*[@id='more']/yt-formatted-string").click() # 카테고리를 크롤링 하기 위해서 더보기를 눌러주자
        time.sleep(3)
        html0 = browser.page_source
        html = BeautifulSoup(html0, 'html.parser')

        # 여기서는 카테고리, 조회수, 좋아요, 싫어요, 영상길이, 동영상 게시일을 크롤링한다 
        category = html.find('div', {'id':'collapsible'}).find('a',{'class':'yt-simple-endpoint'}).text
        # ytd-metadata-row-renderer.style-scope a

        view_count = html.find('span', {'class': 'view-count'}).text # 4

        # 좋아요와 싫어요는 완전히 똑같은 태그에 들어있다.
        # <파이썬 정규 표현식을 지원하는 >re모듈을 사용해보자
        #likes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 5
        #                                                'aria-label':re.compile('좋아요')}).text+'개'
        #dislikes_count = html.find('yt-formatted-string', {'id':'text', 'class':'style-scope ytd-toggle-button-renderer style-text', # 6
        #'aria-label':re.compile('싫어요')}).text+'개'
        likes_count = "0"
        dislikes_count = "0"
        upload_date = html.find('div',{'id':'date'}).find('yt-formatted-string').text # 8
        
        print(i,".","썸네일", thumbnail, "제목:", title, "카테고리",category,"조회수",view_count,"좋아요수",likes_count, "싫어요수",dislikes_count, "게시일", upload_date)
    
    
    finally:    
        thumbnail_list.append(thumbnail)
        title_list.append(title)
        category_list.append(category)
        view_count_list.append(view_count)
        likes_count_list.append(likes_count)
        dislikes_count_list.append(dislikes_count)
        upload_date_list.append(upload_date)

0 . 썸네일 https://i.ytimg.com/vi/_mU-RgFnn4k/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLA2XkBgZfngrdszlYPHJ5BgpUCHXA 제목: 월세 탈출 🎉이사 하는 날, 새로운 집으로. 카테고리 Pauline Maria 조회수 조회수 152,376회 좋아요수 5.2천개 싫어요수 72개 게시일 2020. 1. 4.
1 . 썸네일 https://i.ytimg.com/vi/SXuk_JDhobA/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLB70_Ly7lGf-oiBrD6rUrx9_xYdqQ 제목: Acrylic painting 캔버스에 아크릴화 , 노을지는 바다 그리기 카테고리 Xiuminrmm 조회수 조회수 179,530회 좋아요수 1.4만개 싫어요수 64개 게시일 2019. 12. 7.
2 . 썸네일 https://i.ytimg.com/vi/iq6ZpUVT0Rw/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDjOs17S2d25StEyC__VCKPRee1bg 제목: Winter Outfits : 가을 겨울 룩북, 적은 옷가지로 다양하게 코디하기 카테고리 N F 조회수 조회수 117,041회 좋아요수 3.8천개 싫어요수 63개 게시일 2019. 11. 30.
3 . 썸네일 https://i.ytimg.com/vi/-eH3-XEn55Y/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAErlcKg41Mr2HYgMrE9czOzdaOog 제목: ENG/국내여행, 단풍을 찾아 떠난 전남 구례군. 애견동반 1박2일 여행 카테고리 Esra Kaya 조회수 조회수 68,421회 좋아요수 3.3천개 싫어요수 31개 

32 . 썸네일 https://i.ytimg.com/vi/GzoKNmqm2Dg/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLDAQ1lN0QtRE60VNIXsBnjc6OStCg 제목: ENG/친구들과 생일파티. 1박2일 강릉 여행 카테고리 CattyEve 조회수 조회수 50,549회 좋아요수 1.9천개 싫어요수 28개 게시일 2019. 5. 25.
33 . 썸네일 https://i.ytimg.com/vi/VCTmTU7dCOk/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAs6zNI6rEhaGxoOnrQd7m-siM9Bg 제목: ENG/집순이가 집에서 보내는 평범한 일상 브이로그  *사인회 안내 카테고리 Natalya Timusheva 조회수 조회수 130,293회 좋아요수 3.4천개 싫어요수 52개 게시일 2019. 5. 18.
34 . 썸네일 https://i.ytimg.com/vi/u9kelAGCzwM/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLC9oNHQdZDNzDbjTS_gK1trhuxa_g 제목: ENG/프리랜서로 살기. 4년차 프리랜서가 하는 일. 노토킹 일상#브이로그 카테고리 Thanh Hiền Trần 조회수 조회수 144,465회 좋아요수 5천개 싫어요수 49개 게시일 최초 공개: 2019. 5. 11.
35 . 썸네일 https://i.ytimg.com/vi/tmSwiKHHH50/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBMZUJ0-HIQjZVGHFv1w6jB3oTg7A 제목: ENG/엄마의 심야식당深夜食堂. 면요리 모음 #볶음우동#간장국수 카테고리 Buse Kurt 조회수 조회수 82,593회 좋아요수 2.

64 . 썸네일 https://i.ytimg.com/vi/cPyq1ZQsFDc/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAwrQyL26MEBKTbPxCalKpKRLPKkQ 제목: 베베의 일상/ Bebe's daily life (ENG SUB) 카테고리 Lari Kamelia 조회수 조회수 50,964회 좋아요수 2.1천개 싫어요수 27개 게시일 2018. 10. 6.
65 . 썸네일 https://i.ytimg.com/vi/38BAWD_peeY/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLBODRoGy1lM4ynwM5zpcYugqC1RZw 제목: 셀프 인테리어! 0.3평 창고를 홈카페로 (DIY Interior Design for Room by myself) 카테고리 Lari Kamelia 조회수 조회수 189,393회 좋아요수 4.6천개 싫어요수 56개 게시일 2018. 9. 29.
66 . 썸네일 https://i.ytimg.com/vi/BqEXQOj31i4/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLCd2wBQgmX2xdR3PsvRjrlXhgy1WQ 제목: eng/패션쇼 보고 엄마집에서 주말 보내기 (The last week of September) 카테고리 Lari Kamelia 조회수 조회수 117,920회 좋아요수 3.1천개 싫어요수 37개 게시일 2018. 9. 25.
67 . 썸네일 https://i.ytimg.com/vi/kRCSOPXcmK8/hqdefault.jpg?sqp=-oaymwEZCNACELwBSFXyq4qpAwsIARUAAIhCGAFwAQ==&rs=AOn4CLAp9T9TxgJ9wEvskx0bAUrkj8Wt2g 제목: 집에서 쉬는날의 일상. 집안일하고 블루베리 잼 만들어서 토스트 🍞

In [ ]:
print(len(thumbnail_list),len(video_length_list),len(title_list))

In [ ]:
import copy
video_length_list_copy = copy.deepcopy(video_length_list[:len(thumbnail_list)])
len(video_length_list_copy), len(video_length_list)

In [ ]:
import pandas as pd

insert_data = pd.DataFrame({
                    'thumbnail': thumbnail_list,
                    'title' : title_list,
                    'category': category_list,
                    'view_count': view_count_list,
                    'likes_count': likes_count_list,
                    'dislikes_count':dislikes_count_list,
                    'upload_date': upload_date_list,
                    'video_length' : video_length_list_copy
                    }) 
insert_data

In [ ]:
import openpyxl
insert_data.to_excel("슛뚜.xlsx",encoding="utf8")

In [ ]:
# 소련여자, 박막례, 최고다윽박, ondo온도, 오킹, 오늘의 하늘,튜브몬,박병창
# 슛뚜 미트 난치

In [ ]:
df